1. Import Libraries and Define schema

In [47]:
import pandas as pd
import numpy as np
import re
from textblob import TextBlob

In [48]:
cols = ["sentiment", "tweet_id", "created_at", "query", "user", "text"]
INPUT = "Twitter(X)-Raw_Data-2009.csv"

2. Load safely(avoid single-class chunk)

In [49]:
#Chunk A(start of file)

In [50]:
df_a = pd.read_csv(INPUT, header = None, names = cols, encoding = "latin-1", nrows  = 200000)

In [51]:
df_b = pd.read_csv(INPUT, header=None, names=cols, encoding= "latin-1", skiprows=800000, nrows=200000)

In [52]:
df = pd.concat([df_a, df_b], ignore_index = True)

In [53]:
df["sentiment"] = pd.to_numeric(df["sentiment"], errors= "coerce")

In [54]:
df = df[df["sentiment"].isin([0,4])].copy()

In [55]:
#Gold 2-class label

In [56]:
df["label_gold"] = (df["sentiment"] == 4).astype(int)

In [57]:
df["sentiment"].value_counts(), df["label_gold"].value_counts()

(sentiment
 0    200000
 4    200000
 Name: count, dtype: int64,
 label_gold
 0    200000
 1    200000
 Name: count, dtype: int64)

3.Quick Eda(profiling)

In [58]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   sentiment   400000 non-null  int64
 1   tweet_id    400000 non-null  int64
 2   created_at  400000 non-null  str  
 3   query       400000 non-null  str  
 4   user        400000 non-null  str  
 5   text        400000 non-null  str  
 6   label_gold  400000 non-null  int64
dtypes: int64(3), str(4)
memory usage: 21.4 MB


In [59]:
print(df["label_gold"].value_counts(normalize=True))

label_gold
0    0.5
1    0.5
Name: proportion, dtype: float64


In [60]:
print(df["text"].astype(str).str.len().describe())

count    400000.000000
mean         74.216747
std          36.435371
min           6.000000
25%          44.000000
50%          69.000000
75%         104.000000
max         186.000000
Name: text, dtype: float64


4. Balancing

In [61]:
n_pos = (df["label_gold"]==1).sum()
n_neg = (df["label_gold"]==0).sum()
n = min(n_pos, n_neg, 100000) # up to 100k/class

In [62]:
neg = df[df["label_gold"]==0].sample(n, random_state=42)
pos = df[df["label_gold"]==1].sample(n, random_state=42)

In [63]:
df = pd.concat([neg, pos]).sample(frac=1, random_state=42).reset_index(drop=True)
df["label_gold"].value_counts()

label_gold
1    100000
0    100000
Name: count, dtype: int64

5. Clean text + engineer features

In [64]:
def clean_text(s: str) -> str:
    s = "" if pd.isna(s) else str(s)
    s = s.lower()
    s = re.sub(r"http\S+", " URL ", s)
    s = re.sub(r"@\w+", " @user ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

In [65]:
def parse_created_at(s: str):
    if pd.isna(s):
        return pd.NaT
    
    s = str(s)
    
    # Remove timezone token like PDT, UTC, etc.
    s = re.sub(r"\s[A-Z]{3,4}\s(\d{4})$", r" \1", s)
    
    return pd.to_datetime(
        s,
        format="%a %b %d %H:%M:%S %Y",
        errors="coerce"
    )

In [66]:
df["created_dt"] = df["created_at"].apply(parse_created_at)
df["date"] = df["created_dt"].dt.date.astype(str)
df["hour"] = df["created_dt"].dt.hour

In [67]:
df["text_clean"] = df["text"].apply(clean_text)
df["text_len"] = df["text_clean"].str.len()
df["word_count"] = df["text_clean"].str.split().apply(len)

In [68]:
# Boolean feature flags based on raw tweet text
df["has_url"] = df["text"].str.contains(r"http", na=False)
df["has_mention"] = df["text"].str.contains(r"@\w+", na=False)
df["has_hashtag"] = df["text"].str.contains(r"#\w+", na=False)

In [69]:
df[[
    "text",
    "text_clean",
    "text_len",
    "word_count",
    "has_url",
    "has_mention",
    "has_hashtag"
]].head(5)

,text,text_clean,text_len,word_count,has_url,has_mention,has_hashtag
0,"@burlymulligan yes, theeee Target. it may hap...","@user yes, theeee target. it may happen again ...",55,10,False,True,False
1,Should I be pointing Twhirl at FF? How would I...,should i be pointing twhirl at ff? how would i...,117,24,False,False,False
2,http://twitpic.com/3l7l6 - Sunday bake with au...,URL - sunday bake with auntie wendy new york c...,56,11,True,False,False
3,@MCRsavedMilife whyd you leave?,@user whyd you leave?,21,4,False,True,False
4,I miss baby Lambert.,i miss baby lambert.,20,4,False,False,False


6. 3-Class Sentiment (Negative / Neutral / Positive)

In [70]:
def polarity_textblob(s: str) -> float:
    try:
        return float(TextBlob(s).sentiment.polarity)
    except Exception:
        return 0.0

In [71]:
def sentiment_3class(p: float) -> str:
    if p >= 0.05: return "Positive"
    if p <= -0.05: return "Negative"
    return "Neutral"

In [72]:
df["polarity"] = df["text_clean"].apply(polarity_textblob)
df["sentiment_3class"] = df["polarity"].apply(sentiment_3class)
df["sentiment_3class"].value_counts(normalize=True)

sentiment_3class
Positive    0.42654
Neutral     0.37790
Negative    0.19556
Name: proportion, dtype: float64

7. Create export tables for Power BI

In [73]:
import os

In [74]:
OUT_DIR = "powerbi_out_3class"
os.makedirs(OUT_DIR, exist_ok=True)

In [75]:
# FACT table (Power BI-friendly, no heavy raw text)
fact = df[[
    "tweet_id","created_dt","date","hour","user",
    "label_gold",                # optional reference (2-class)
    "polarity","sentiment_3class",
    "has_url","has_mention","has_hashtag",
    "text_len","word_count"
]].copy()

In [76]:
# Optional: small text table for drill-through (keep small)
tweet_text_sample = df[["tweet_id","text_clean"]].sample(min(10000, len(df)), random_state=42)

In [77]:
# Daily summaries for fast visuals
daily_summary = (df.groupby(["date","sentiment_3class"], as_index=False)
                 .agg(tweet_count=("tweet_id","count"),
                      avg_polarity=("polarity","mean"),
                      avg_text_len=("text_len","mean"),
                      avg_word_count=("word_count","mean")))

In [78]:
# Hourly summaries
hourly_summary = (df.groupby(["hour","sentiment_3class"], as_index=False)
                  .agg(tweet_count=("tweet_id","count"),
                       avg_polarity=("polarity","mean")))

In [79]:
fact.to_csv(f"{OUT_DIR}/fact_tweets_3class.csv", index=False, encoding="utf-8")
tweet_text_sample.to_csv(f"{OUT_DIR}/tweet_text_sample.csv", index=False, encoding="utf-8")
daily_summary.to_csv(f"{OUT_DIR}/daily_summary_3class.csv", index=False, encoding="utf-8")
hourly_summary.to_csv(f"{OUT_DIR}/hourly_summary_3class.csv", index=False, encoding="utf-8")

In [80]:
print("Exported files:", os.listdir(OUT_DIR))
print("Fact rows:", len(fact))

Exported files: ['daily_summary_3class.csv', 'fact_tweets_3class.csv', 'hourly_summary_3class.csv', 'tweet_text_sample.csv']
Fact rows: 200000
